# VAE

In [1]:
import sys
import os

sys.path.append( '..' )
from VAE import *
from VAE_smac import *
from helpers.pc_stats import *

In [6]:
a = "abcd"
a[:3]

'abc'

## Parameter definition

In [2]:
data_dir  = os.path.normpath(os.path.join(os.getcwd(), "../../runs/FIA/Com8_grown_together/oms"))
run_dir = os.path.normpath(os.path.join(os.getcwd(), "../../runs/VAE/hyperparameter_optimization"))
results_dir = os.path.normpath(os.path.join(os.getcwd(), "../../runs/VAE/results"))
test_configuration = False
overwrite = False
verbosity = 1
framework = "pytorch_jupyter"
outdir = Path(os.path.normpath(os.path.join(run_dir, f"smac_vae_{framework}")))

# Logging (time and steps)
last_timestamp = time.time()
step = 0
runtimes = {}


time_step(message="Setup loaded", verbosity=verbosity)

Setup loaded (0.010263919830322266s)


In [3]:
X = read_data(data_dir, verbosity=verbosity)

configuration_space = ConfigurationSpace(name="LD", seed=42)
hyperparameters = [
    Constant(       "original_dim",             X.shape[1]),
    Float(          "input_dropout",            (0.0, 0.5), default=0.25),
    Integer(        "intermediate_layers",      (1, 5), default=2),
    Integer(        "intermediate_dimension",   (1, 20), log=True, default=4),
    Categorical(    "intermediate_activation",  ["relu", "selu", "tanh", "leakyrelu"], default="leakyrelu"),
    Integer(        "latent_dimension",         (1, 10), log=False, default=2),
    Categorical(    "solver",                   ["nadam"], default="nadam"),
    Float(          "learning_rate",            (1e-4, 1e-2), log=True, default=1e-3)
]
configuration_space.add_hyperparameters(hyperparameters)
forbidden_clauses = [
    ForbiddenGreaterThanRelation(configuration_space["latent_dimension"], configuration_space["intermediate_dimension"])
]
configuration_space.add_forbidden_clauses(forbidden_clauses)
if verbosity > 0: 
    print(f"Configuration space defined with estimated {configuration_space.estimate_size()} possible combinations.\n")

Data loaded (2.1655523777008057s)
Configuration space defined with estimated inf possible combinations.



## Optimization definition

In [4]:
fia_vae_tune = FIA_VAE_tune( X, test_size=0.2, configuration_space=configuration_space, model_builder=FIA_VAE,
                                batch_size=64, log_dir=os.path.join(outdir, "log"), verbosity=verbosity, gpu=False )

In [5]:
fia_vae = FIA_VAE(configuration_space.get_default_configuration())

In [6]:
print(configuration_space.get_default_configuration())
fia_vae.summary()
#fia_vae.fit(x=X, y=X, batch_size=64, epochs=10, verbose="auto", validation_split=0.2)

Configuration(values={
  'input_dropout': 0.25,
  'intermediate_activation': 'leakyrelu',
  'intermediate_dimension': 4,
  'intermediate_layers': 2,
  'latent_dimension': 2,
  'learning_rate': 0.001,
  'original_dim': 825000,
  'solver': 'nadam',
})


Model: "fia_vae"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_intermediate            │ ?                      │     3,300,004 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent_mu (Dense)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent_sigma (Dense)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent_reparametrization        │ ?                      │   0 (unbuilt) │
│ (Sampling)                      │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder (Sequential)            │ ?                      │     4,125,012 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,425,016 (28.32 MB)

 Trainable params: 7,425,016 (28.32 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
scenario = Scenario( fia_vae_tune.configuration_space, deterministic=True,
                     n_trials=20, min_budget=2, max_budget=100,
                     n_workers=1, output_directory=outdir,
                     walltime_limit=np.inf, cputime_limit=np.inf, trial_memory_limit=None )   # Max RAM in Bytes (not MB)
                    
initial_design = MultiFidelityFacade.get_initial_design(scenario, n_configs=10)
intensifier = Hyperband(scenario, incumbent_selection="highest_budget")
facade = MultiFidelityFacade( scenario, fia_vae_tune.train, 
                              initial_design=initial_design, intensifier=intensifier,
                              overwrite=overwrite, logging_level=30-verbosity*10 )
time_step(message=f"SMAC defined. Overwriting: {overwrite}", verbosity=verbosity)

[INFO][abstract_initial_design.py:147] Using 5 initial design configurations and 0 additional configurations.
[INFO][smbo.py:497] Continuing from previous run.
SMAC defined. Overwriting: False (1.8681108951568604s)


### Search

In [10]:
incumbent = run_optimization(facade=facade, smac_model=fia_vae_tune, verbose_steps=10, verbosity=3)

Starting search:


  0%|          | 0/10 [00:00<?, ?it/s]

Configuration: {'input_dropout': 0.22677134133903443, 'intermediate_activation': 'tanh', 'intermediate_dimension': 12, 'intermediate_layers': 5, 'latent_dimension': 9, 'learning_rate': 0.00010554269228116391, 'original_dim': 825000, 'solver': 'nadam'}
Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 25s 25s/step - kl_loss: 1.7881e-07 - loss: 0.0026 - reconstruction_loss: 0.0026 - val_kl_loss: 3.7849e-06 - val_loss: 0.0025 - val_reconstruction_loss: 0.0025
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 24s 24s/step - kl_loss: 1.5169e-05 - loss: 0.0025 - reconstruction_loss: 0.0025 - val_kl_loss: 1.7583e-06 - val_loss: 0.0023 - val_reconstruction_loss: 0.0023
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 25s 25s/step - kl_loss: 6.3479e-06 - loss: 0.0024 - reconstruction_loss: 0.0024 - val_kl_loss: 1.1921e-07 - val_loss: 0.0021 - val_reconstruction_loss: 0.0021
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 702ms/step - kl_loss: 2.3842e-07 - loss: 0.0023 - reconstruction_loss: 0.0023


 10%|█         | 1/10 [01:26<12:55, 86.19s/it]

Configuration: {'input_dropout': 0.364995281212029, 'intermediate_activation': 'relu', 'intermediate_dimension': 3, 'intermediate_layers': 1, 'latent_dimension': 2, 'learning_rate': 0.00010890393916830513, 'original_dim': 825000, 'solver': 'nadam'}
Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 24s 24s/step - kl_loss: 0.0000e+00 - loss: 8.6256e-04 - reconstruction_loss: 8.6256e-04 - val_kl_loss: 3.2783e-07 - val_loss: 6.0616e-04 - val_reconstruction_loss: 6.0583e-04
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 28s 28s/step - kl_loss: 1.2815e-06 - loss: 6.4354e-04 - reconstruction_loss: 6.4226e-04 - val_kl_loss: 0.0000e+00 - val_loss: 8.4552e-04 - val_reconstruction_loss: 8.4552e-04
Epoch 3/3


 10%|█         | 1/10 [02:36<23:31, 156.81s/it]


KeyboardInterrupt: 

In [9]:
from keras import backend

print(backend.backend())

torch
